In [1]:
import sys
import os
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot as plt
from numpy import pi, matmul,sqrt,dot,array,zeros,cos,sin,pi,arccos
from torch.utils.data import DataLoader
from func import OR,heatplot,misorientation,ipfread,negsample,properties
from func2 import create_dataset
from Class import Data,Cluster,Dataset,CNN49ag,CNN49bg
import torch.nn.functional as F
import matplotlib.patches as patches


In [9]:
header=['ND面硬度(Hv)', 'TD面硬度(Hv)', 'UTS strain(%)', 'UTS stress(MPa)','Total Elongation(%)', 'E(GPa)', 'Ys(Mpa)', 'C2', 'Si1', 'Mn3', 'P1','Cu5', 'Cr', 'Ti4', 'Al7', 'FE%']
pic_size=500
sam_size=49
prop_dic = torch.load("data/properties.pkl")
train=create_dataset("train",feature=feature,prop_dic=prop_dic,pic_size=pic_size,sam_size=sam_size)
val=create_dataset("val",feature=feature,prop_dic=prop_dic,pic_size=pic_size,sam_size=sam_size)



In [39]:
class CNN49ag(nn.Module):
    def __init__(self,in_channel,comp):
        super(CNN49ag, self).__init__()
        self.batchnorm = nn.BatchNorm2d(in_channel)
        self.relu = nn.ReLU() # activation
        self.maxpool = nn.MaxPool2d(kernel_size=2) 
        self.cnn1 = nn.Conv2d(in_channels=in_channel, out_channels=12, kernel_size=2, stride=1, padding=0) 
        self.cnn2 = nn.Conv2d(in_channels=12, out_channels=18, kernel_size=3, stride=1, padding=0) 
        self.cnn3 = nn.Conv2d(in_channels=18, out_channels=24, kernel_size=3, stride=1, padding=0) 
        self.fc1 = nn.Linear(24 * 9 * 9+comp, 2) 
        self.softmax = nn.Softmax(1)
    def forward(self, x,p):
        # Convolution 1 49
        out = self.batchnorm(x)
        out = self.cnn1(x) # 48
        out = self.relu(out)
        out = self.maxpool(out) #24
        out = self.cnn2(out) #22
        out = self.relu(out) 
        out = self.maxpool(out) #11
        out = self.cnn3(out) #9
        out = self.relu(out)
        out = out.view(out.size(0), -1)
        if len(p)>1:
            out = torch.cat([out,p],1)
        out = self.fc1(out)
        out = self.softmax(out)
        return out
comp=len(train[0][1])
in_channel=train[0][0].shape[0]
model=CNN49ag(in_channel,comp)
train_loader= DataLoader(train, batch_size=64, shuffle=True,  num_workers=0, drop_last=True )
val_loader= DataLoader(val, batch_size=64, shuffle=True,  num_workers=0,  drop_last=False )
Loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
max_acc = 0.7
epoch = 200
for ep in range(epoch):
    for batch_ndx, sample in enumerate(train_loader):
        optimizer.zero_grad()
        X,P,Y,_= sample
        output=model(X,P)
        loss = Loss(output,Y)
        loss.backward()
        optimizer.step()
#         print("train ",ep,loss.item())
    model.eval()
    cum=0
    loss=0
    for batch_ndx, sample in enumerate(val_loader):
        X,P,Y,_= sample
        output=model(X,P)
        loss+=Loss(output,Y)
        predict = torch.max(output, 1)[1]
        cum+=np.sum((Y == predict).cpu().numpy())
    acc=cum/len(val)
    if acc>max_acc:
        max_acc=acc
        min_loss = loss.item()
        print(ep,min_loss)
        print("acc= ",cum,"/",len(val))
#         torch.save(model,"model/%.3f_%s_%s.pkl"%(acc,type(model).__name__,"_".join(feature)))


24 1.2039592266082764
acc=  77 / 108
30 1.1884856224060059
acc=  78 / 108
37 1.1676735877990723
acc=  79 / 108
39 1.1728630065917969
acc=  80 / 108
103 1.1506340503692627
acc=  81 / 108
